In [44]:
import os
import wget
import zlib
import json
import pprint
import urllib
import requests
from functools import partial
from bs4 import BeautifulSoup
from selenium import webdriver

In [6]:
def request(url, headers=None):
    if headers is None:
        headers = {
            'Content-Type': 'text/html',
            'Accept-Encoding': 'identity'
        }
    response = requests.get(url, headers=headers)
    return response

In [61]:
# for use with filter method
def imagefilter(image, exclude_words):
    for w in exclude_words:
        if w in image:
            return False
    return True

In [87]:
# for use with filter method
def is_valid_image(image, exclude_words):
    if 'tumblr_' not in image:
        return False

    for w in exclude_words:
        if w in image:
            return False

    return True

In [7]:
baseurl = 'https://www.tumblr.com/search'

In [8]:
search = 'cowboy bebop'
url = urllib.parse.quote_plus(search)
url = f'{baseurl}/{url}'
url

'https://www.tumblr.com/search/cowboy+bebop'

In [9]:
VALID_EXTS = ['.png', '.jpg', '.gif', '.gifv']

In [81]:
EXCLUDE_WORDS = ['assets', 'static', '16x16', '64x64', '48x48']

In [69]:
response = request('https://gahdamnpunk.tumblr.com/post/185736435074/the-critical-thinking-the-self-awarenesstaste')
soup = BeautifulSoup(response.content, 'html.parser')
# Get all images
images = []
aimages = soup.find_all('img')
for image in aimages:
    image = image.get('src')
    ext = os.path.splitext(image)[-1]
    if ext in VALID_EXTS and is_valid_image(image, EXCLUDE_WORDS):
        images.append(image)

In [88]:
response = request(url)
soup = BeautifulSoup(response.content, 'html.parser')


scraped_data = []
articles = soup.find_all('article')
for i, article in enumerate(articles, start=1):
    print(f'Processing {i}/{len(articles)}')
    data = json.loads(article.get('data-json'))
    post_url = data['share_popover_data']['post_url']
    response = request(post_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Get image tags
    try:
        tags = soup.find('meta', {'name': 'keywords'}).get('content').split(',')
    except AttributeError:
        print('Page has no visible tags - skipping')
        tags = []
        
    # Get all images
    images = []
    aimages = soup.find_all('img')
    for image in aimages:
        image = image.get('src')
        ext = os.path.splitext(image)[-1]
        if ext in VALID_EXTS and is_valid_image(image, EXCLUDE_WORDS):
            images.append(image)

    scraped_data.append({'url': post_url, 'tags': tags, 'images': images})

Processing 1/20
Processing 2/20
Processing 3/20
Processing 4/20
Processing 5/20
Processing 6/20
Processing 7/20
Processing 8/20
Processing 9/20
Processing 10/20
Processing 11/20
Processing 12/20
Processing 13/20
Processing 14/20
Processing 15/20
Processing 16/20
Processing 17/20
Processing 18/20
Processing 19/20
Processing 20/20


In [89]:
scraped_data

[{'url': 'https://kikuhikou.tumblr.com/post/186474360671/classics-series-iconic-anime-of-the-90s',
  'tags': ['allanimanga',
   'graphicsources',
   'fyanimegifs',
   'neon genesis evangelion',
   'cowboy bebop'],
  'images': ['https://64.media.tumblr.com/a4bd5eb4f1a65dcabc02176ac69edc9d/tumblr_puys87ZJnE1tkezr8o7_r1_500.gifv',
   'https://64.media.tumblr.com/4daf8fd752b1808d72bdddd091415b7a/tumblr_puys87ZJnE1tkezr8o5_r1_500.gifv',
   'https://64.media.tumblr.com/b61bdb65314d3985b9fb579d52010e18/tumblr_puys87ZJnE1tkezr8o4_r1_500.gifv',
   'https://64.media.tumblr.com/b4b9404369f38a3835c5d9af6bcde94f/tumblr_puys87ZJnE1tkezr8o2_500.gifv',
   'https://64.media.tumblr.com/8a0533f604288d0efc9222d68cc06faa/tumblr_puys87ZJnE1tkezr8o6_r1_500.gifv',
   'https://64.media.tumblr.com/0f59c56cca8c5e72c8119ea4042cb120/tumblr_puys87ZJnE1tkezr8o8_r1_500.gifv',
   'https://64.media.tumblr.com/0e09950eb20279c3459a48edbf691f6e/tumblr_puys87ZJnE1tkezr8o9_r1_500.gifv',
   'https://64.media.tumblr.com/79b6f

In [86]:
# base = 'images'
# os.makedirs(base, exist_ok=True)
# for photo in photos:
#     wget.download(photo, out=base)